## Sentiment analyse twitter

In [1]:
import pandas as pd  
import numpy as np
import matplotlib.pyplot as plt
import re

### We load an explore our labeled dataset 

labeld training set can be download from:http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip

In [2]:
##This chunck is for reading in the train and test data, including a little bit of exploratory.

cols = ['sentiment','id','date','query_string','user','text']
df = pd.read_csv("training.1600000.processed.noemoticon.csv",header=None, names=cols,encoding="ISO-8859–1")
# above line will be different depending on where you saved your data, and your file name
df.head()
df.drop(['id','date','query_string','user'],axis=1,inplace=True) ## Drop the rows that we dont need
df[df.sentiment == 4].head(10) ## Show some rows of sentiment 4 , looks like 4 is positive


,sentiment,text
800000,4,I LOVE @Health4UandPets u guys r the best!!
800001,4,im meeting up with one of my besties tonight! ...
800002,4,"@DaRealSunisaKim Thanks for the Twitter add, S..."
800003,4,Being sick can be really cheap when it hurts t...
800004,4,@LovesBrooklyn2 he has that effect on everyone
800005,4,@ProductOfFear You can tell him that I just bu...
800006,4,@r_keith_hill Thans for your response. Ihad al...
800007,4,"@KeepinUpWKris I am so jealous, hope you had a..."
800008,4,"@tommcfly ah, congrats mr fletcher for finally..."
800009,4,@e4VoIP I RESPONDED Stupid cat is helping me ...


In [3]:
df[df.sentiment == 0].head(10) ## Show some rows of sentiment 0, looks like 0 is negative

,sentiment,text
0,0,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,is upset that he can't update his Facebook by ...
2,0,@Kenichan I dived many times for the ball. Man...
3,0,my whole body feels itchy and like its on fire
4,0,"@nationwideclass no, it's not behaving at all...."
5,0,@Kwesidei not the whole crew
6,0,Need a hug
7,0,@LOLTrish hey long time no see! Yes.. Rains a...
8,0,@Tatiana_K nope they didn't have it
9,0,@twittera que me muera ?


In [4]:
df['sentiment'] = df['sentiment'].map({0: 0, 4: 1}) ## Map indicater 4 to 1
df.sentiment.value_counts() ## Check how much labeled is 800k sentiment 4 and 800k sentiment 0

1    800000
0    800000
Name: sentiment, dtype: int64

## We load our untagged dataset

In [5]:
%%time
# Verander in de regel hieronder de locatie van de data
tweets = pd.read_json("mini_twitter_geotagged_clean.json", orient='records', lines=True)


Wall time: 20 s


In [6]:
tweets['ID'] = tweets.index ## Performance reasons and slicing for myself
tweets = tweets[['ID','id','CleanText','text','entities','in_reply_to_status_id', 'quoted_status_id']]
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id
0,0,764039733076897792,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\n...,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'hashtags': [{'text': 'NOJUSTICE', 'indices':...",NaN,NaN
1,1,764039812479225856,he will do in one year all the things you shou...,@HillaryClinton he will do in one year all the...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",7.626233e+17,NaN
2,2,764039849850482689,clear deliberately throwing this racein 2007 h...,#CNN #newday clear #Trump deliberately throwin...,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4...",NaN,NaN
3,3,764039917924069376,you wouldnt recognize a lie if it came from y...,"@realDonaldTrump, you wouldn't recognize a lie...","{'hashtags': [{'text': 'NeverTrump', 'indices'...",NaN,7.639507e+17
4,4,764039926161604608,Kid you know suing someone Thats the most beau...,"""Kid, you know, suing someone? Thats the most ...","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN


In [7]:
# Create a new coloumn that contains a list with all the mentions used in the tweet
def mentionfinder(row):
    mentions = re.findall('(?<![@\w])@(\w{1,25})', row['text'])
    mentions = [x.lower() for x in mentions]
    #mentions = tuple(mentions)
    return mentions

tweets['Mentions'] = tweets.apply(lambda row: mentionfinder(row), axis=1)

In [8]:
# Create a new coloumn that contains a list with all the hastags used in the tweet
def hashtagfinder(row):
    hashtags = re.findall('(?:(?<=\s)|(?<=^))#.*?(?=\s|$|\'|\-|[.,\/#!$%\^&\*;:{}=\-_`~()])', row['text']) 
    hashtags = [x.lower() for x in hashtags]
    return hashtags

tweets['Hashtags'] = tweets.apply(lambda row: hashtagfinder(row), axis=1)

In [9]:
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,0,764039733076897792,\n\n\nALL IN COLLUSION TOGETHER \n\n\n\n\n\n\n...,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'hashtags': [{'text': 'NOJUSTICE', 'indices':...",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,1,764039812479225856,he will do in one year all the things you shou...,@HillaryClinton he will do in one year all the...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",7.626233e+17,NaN,[hillaryclinton],[]
2,2,764039849850482689,clear deliberately throwing this racein 2007 h...,#CNN #newday clear #Trump deliberately throwin...,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4...",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,3,764039917924069376,you wouldnt recognize a lie if it came from y...,"@realDonaldTrump, you wouldn't recognize a lie...","{'hashtags': [{'text': 'NeverTrump', 'indices'...",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,4,764039926161604608,Kid you know suing someone Thats the most beau...,"""Kid, you know, suing someone? Thats the most ...","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


In [10]:
len(tweets)

46811

## Cleansing the tagged train data
The untagged dataset is already cleansed in the Preprocessing.ipynb
* Removal of http and www links
* Removal of hashtags #
* Removal of multiple spaces

In addition we delete enter and tab encoding. And words that are smaller than 3 letters.

In [11]:
def clean_string(row):
    # remove http and www links from string
    rem_hl = re.sub(r'(http\S+) | (www\S+) | (https\S+)', '', row['text'])
    
    # remove @.. from string
    rem_at_hl = re.sub(r'@(\w+) |@(\w+)', '', rem_hl)
    
    # remove hashtags #.. from string
    rem_hash_at_hl = re.sub(r'#(\w+) |@(\w+)', '', rem_at_hl)

    rem_inter = re.sub(r'[^\w\s]','', rem_hash_at_hl)
    
    # remove multiple spaces
    rem_space = re.sub(' +',' ',rem_inter)
    #print (re.search('[a-zA-Z]', rem_space))
    if (re.search('[a-zA-Z]', rem_space) == None):
        return("emptyStringRetured")
    else:
        return (rem_space)


In [12]:
df['text'] = df.apply(lambda row: clean_string(row), axis=1)

In [13]:
tweets['CleanText'] = tweets['CleanText'].replace('\n','')
tweets['CleanText'] = tweets['CleanText'].replace('\t','')

df['text'] = df['text'].replace('\n','')
df['text'] = df['text'].replace('\t','')

tweets['CleanText'] = tweets['CleanText'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
df['text'] = df['text'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))


tweets['CleanText'] =tweets['CleanText'].str.lower()
df['text'] = df['text'].str.lower()

In [14]:
np.sum(df.isnull().any(axis=1))
df.isnull().any(axis=0)

sentiment    False
text         False
dtype: bool

In [15]:
## This chuck is ment for further exploration
# df.isnull().any(axis=0)
# np.sum(df.isnull().any(axis=1))
# tweets.isnull().any(axis=0)
# np.sum(tweets.isnull().any(axis=1))
df.head() 
tweets.head()

,ID,id,CleanText,text,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,0,764039733076897792,collusion together httpstco5gmnzq40v3,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN CO...,"{'hashtags': [{'text': 'NOJUSTICE', 'indices':...",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,1,764039812479225856,will year things should have done eight,@HillaryClinton he will do in one year all the...,"{'hashtags': [], 'urls': [], 'user_mentions': ...",7.626233e+17,NaN,[hillaryclinton],[]
2,2,764039849850482689,clear deliberately throwing this racein 2007 k...,#CNN #newday clear #Trump deliberately throwin...,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4...",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,3,764039917924069376,wouldnt recognize came from your mouth they co...,"@realDonaldTrump, you wouldn't recognize a lie...","{'hashtags': [{'text': 'NeverTrump', 'indices'...",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,4,764039926161604608,know suing someone thats most beautiful thing ...,"""Kid, you know, suing someone? Thats the most ...","{'hashtags': [], 'urls': [], 'user_mentions': ...",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


## Word counts in labeld data

In [16]:
from sklearn.feature_extraction.text import CountVectorizer
cvec = CountVectorizer()
cvec.fit(df.text)

neg_doc_matrix = cvec.transform(df[df.sentiment == 0].text)
pos_doc_matrix = cvec.transform(df[df.sentiment == 1].text)
neg_tf = np.sum(neg_doc_matrix,axis=0)
pos_tf = np.sum(pos_doc_matrix,axis=0)
neg = np.squeeze(np.asarray(neg_tf))
pos = np.squeeze(np.asarray(pos_tf))
term_freq_df = pd.DataFrame([neg,pos],columns=cvec.get_feature_names()).transpose()

In [17]:
term_freq_df.columns = ['negative', 'positive']
term_freq_df['total'] = term_freq_df['negative'] + term_freq_df['positive']
term_freq_df.sort_values(by='total', ascending=False).iloc[:10]

,negative,positive,total
have,82462,62039,144501
that,72583,68739,141322
just,63178,61874,125052
with,49919,64926,114845
this,52087,40890,92977
good,28551,60840,89391
like,40639,37106,77745
dont,44916,22009,66925
your,20752,44244,64996
today,36189,28400,64589


## Sentiment analyse --Step 1 splitting the data

In [18]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['text'].values, df['sentiment'].values, test_size=0.2)

## Sentiment analyse -- Step 2 tokanize

In [19]:
import sklearn
vect = CountVectorizer() ## This is the tokanizer
tf_train = vect.fit_transform(X_train)
tf_test = vect.transform(X_test)


In [20]:
tf_train ##1280k

<1280000x396524 sparse matrix of type '<class 'numpy.int64'>'
	with 8556515 stored elements in Compressed Sparse Row format>

In [21]:
tf_test ##320k

<320000x396524 sparse matrix of type '<class 'numpy.int64'>'
	with 2069340 stored elements in Compressed Sparse Row format>

In [22]:
## This chunk is ment to check if the porportions are still right
sum(y_train) ## 640107   is the half of 1280 k in the train set
sum(y_test) ## 159893   is the half of 320k in the test set

159540

## Sentiment analyse -- Step 3 Building a classifier

In [23]:
# p = sum of all feature count vectors with label 1
p = tf_train[y_train==1].sum(0) + 1
# q = sum of all feature count vectors with label 0
q = tf_train[y_train==0].sum(0) + 1
# Notice that we add 1 to both count vectors to ensure that every token appear at least one time in each class.

# The log-count ratio r is:
r = np.log((p/p.sum()) / (q/q.sum()))
# And b:
b = np.log(len(p) / len(q))

# Predictions

# With calculated coefficients we can now generate predictions on test set. Since we are trying to fit a linear classifier, the form of the linear equation is:

# y = mx + b
pre_preds = tf_test @ r.T + b
preds = pre_preds.T > 0
accuracy = (preds == y_test).mean()
accuracy


0.758965625

The accuracy is 75.8 % meaning that the algorithm classisfy the correct label in 3 out of the 44 labels.

In [24]:
## Visualize the data
test_outcome = pd.DataFrame({'text':X_test, 'Positive':np.asarray(preds)[0]})
test_outcome.head(15)

,text,Positive
0,misses kitty just broke down want kitty back,False
1,actually thinking about deleting pics there st...,False
2,goood morning work only half hour late have ni...,True
3,momma misses,False
4,just left kristins what wiill night bring,False
5,excuses getting keep popping,True
6,what hell wrong with seem think that doing eve...,False
7,thats what what,True
8,hell yeah agree,True
9,love this tune fuck study asot400,True


## Sentiment analyse -- Step 4 Score out on our dataset

In [25]:
# Vectorize the unseen data
unseen = vect.transform(tweets.CleanText)
pre_unseen = unseen @ r.T + b
predictions_unseen = pre_unseen.T > 0

In [26]:
##Visualize the data
pd.options.display.max_rows  ## This is for showing all the text in the column (otherwise it will be half....)
pd.set_option('display.max_colwidth', -1)

Predictions_unseen_df = pd.DataFrame({'text':tweets.text, 'Positive':np.asarray(predictions_unseen)[0]})
Predictions_unseen_df.head(20)


,text,Positive
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,True
1,@HillaryClinton he will do in one year all the things you should have done in eight,True
2,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion",False
3,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm",False
4,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s",True
5,"@HillaryClinton you ARE the co-founder of ISIS, you crooked, evil, lying, witch. How can you live with yourself?",False
6,@Geraldanthro @NeilTurner_ @realDonaldTrump want to do a comparison try maimed Vets pre &amp; post Iraq pullout. Bar graph that. @washingtonpost,False
7,@mike4193496 @realDonaldTrump I TOTALLY CONCUR!! This Election is just CRA CRA n Corruption in our Gov is Mind Blowing!! Trump= Last Hope!!!,False
8,"@realDonaldTrump @elsolarverde What issues? Your idiot claim that she ""founded"" ISIS? Trump will go to Hell for lying &amp; stealing. Shame!",False
9,Can't stand @HillaryClinton or @realDonaldTrump?Take a look. They can win...don't settle. #15for15 #TeamGov #YouIn https://t.co/YK336aaH98,False


In [27]:
## This is a check if we have still all the data (46811) in the dataframe
len(Predictions_unseen_df) #46811
sum(Predictions_unseen_df.Positive) ## Amount of positive == 24217

24196

## Write back the dataset to share

In [28]:
# tweets = pd.read_json("C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean.json", orient='records', lines=True)
# tweets["Positive"] = np.asarray(predictions_unseen)[0]
# tweets.head()
tweets2 = tweets
# tweets.to_json('C:\\Users\\Jeffr\\Downloads\\twitter_geotagged_clean_including_sentiment.json', orient='records', lines=True)

# Vanaf hier begint de selectie op trump of hillary

In [29]:
full = pd.merge(Predictions_unseen_df,tweets2, how='left', left_on='text', right_on='text')

In [30]:
full.head()

,text,Positive,ID,id,CleanText,entities,in_reply_to_status_id,quoted_status_id,Mentions,Hashtags
0,@BarackObama \n@FBI\n@LORETTALYNCH \nALL IN COLLUSION TOGETHER \n\n#NOJUSTICE \n\n@realDonaldTrump \n#TrumpPence \n\nhttps://t.co/5GMNZq40V3,True,0,764039733076897792,collusion together httpstco5gmnzq40v3,"{'hashtags': [{'text': 'NOJUSTICE', 'indices': [62, 72]}, {'text': 'TrumpPence', 'indices': [93, 104]}], 'urls': [{'url': 'https://t.co/5GMNZq40V3', 'expanded_url': 'http://www.zerohedge.com/news/2016-08-11/justice-department-prevented-fbi-probe-clinton-foundation-reporters-slam-state-depar', 'display_url': 'zerohedge.com/news/2016-08-1…', 'indices': [107, 130]}], 'user_mentions': [{'screen_name': 'BarackObama', 'name': 'Barack Obama', 'id': 813286, 'id_str': '813286', 'indices': [0, 12]}, {'screen_name': 'FBI', 'name': 'FBI', 'id': 17629860, 'id_str': '17629860', 'indices': [14, 18]}, {'screen_name': 'LorettaLynch', 'name': 'AG Loretta Lynch', 'id': 3290070855, 'id_str': '3290070855', 'indices': [19, 32]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [75, 91]}], 'symbols': []}",NaN,NaN,"[barackobama, fbi, lorettalynch, realdonaldtrump]","[#nojustice, #trumppence]"
1,@HillaryClinton he will do in one year all the things you should have done in eight,True,1,764039812479225856,will year things should have done eight,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'HillaryClinton', 'name': 'Hillary Clinton', 'id': 1339835893, 'id_str': '1339835893', 'indices': [0, 15]}], 'symbols': []}",7.626233e+17,NaN,[hillaryclinton],[]
2,"#CNN #newday clear #Trump deliberately throwing this race,in 2007 he knew that #ISIS and destabilization of Mideast started w/Iraq invasion",False,2,764039849850482689,clear deliberately throwing this racein 2007 knew that destabilization mideast started wiraq invasion,"{'hashtags': [{'text': 'CNN', 'indices': [0, 4]}, {'text': 'newday', 'indices': [5, 12]}, {'text': 'Trump', 'indices': [19, 25]}, {'text': 'ISIS', 'indices': [79, 84]}], 'urls': [], 'user_mentions': [], 'symbols': []}",NaN,NaN,[],"[#cnn, #newday, #trump, #isis]"
3,"@realDonaldTrump, you wouldn't recognize a lie if it came from your own mouth, and they do continually. #NeverTrump https://t.co/pKSQM8yikm",False,3,764039917924069376,wouldnt recognize came from your mouth they continually nevertrump,"{'hashtags': [{'text': 'NeverTrump', 'indices': [104, 115]}], 'urls': [{'url': 'https://t.co/pKSQM8yikm', 'expanded_url': 'https://twitter.com/realdonaldtrump/status/763950742755434498', 'display_url': 'twitter.com/realdonaldtrum…', 'indices': [116, 139]}], 'user_mentions': [{'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [0, 16]}], 'symbols': []}",NaN,7.639507e+17,[realdonaldtrump],[#nevertrump]
4,"""Kid, you know, suing someone? Thats the most beautiful thing 1 human being could do to another human being"" @funnyordie @realDonaldTrump😂💩s",True,4,764039926161604608,know suing someone thats most beautiful thing human being could another human being,"{'hashtags': [], 'urls': [], 'user_mentions': [{'screen_name': 'funnyordie', 'name': 'Funny Or Die', 'id': 15693493, 'id_str': '15693493', 'indices': [109, 120]}, {'screen_name': 'realDonaldTrump', 'name': 'Donald J. Trump', 'id': 25073877, 'id_str': '25073877', 'indices': [121, 137]}], 'symbols': []}",NaN,NaN,"[funnyordie, realdonaldtrump]",[]


In [31]:
Hillary = ['#dumptrump','#nevertrump', '#imwithher']
Neutral = ['#trumppence16', '#hillaryclinton', '#hillary', '#donaldtrump', '#trump']
NeutralTrump = ['#trumppence16', '#donaldtrump', '#trump', '#donaldtrump\'s', '#trump\'s']
NeutralHillary = ['#hillaryclinton', '#hillary', '#hillaryclinton\'s']
Trump = ['#maga', '#crookedhillary', '#neverhillary']
allhash = ['@HillaryClinton', '#maga', '#trumppence16', '#hillaryclinton', '#hillary', '#crookedhillary', '#donaldtrump', '#dumptrump', '@realDonaldTrump', '#nevertrump', '#imwithher', '#neverhillary', '#trump']

In [32]:
import math

In [33]:
"""
Assumption: Everyone that tweets negative about Hillary is for Trump 
(and the other way around).
"""
def trumphillary(row):
    #print(row)
    # HASHTAGS
    # First checking if there are hashtags used that are person specific
    if len(row['Hashtags']) != 0:
        if ('#dumptrump' in row['Hashtags'] or '#nevertrump' in row['Hashtags'] or '#imwithher' in row['Hashtags']) and ('#maga' in row['Hashtags'] or '#crookedhillary' in row['Hashtags'] or '#neverhillary' in row['Hashtags']):
            return 'Both'
        for i in range(len(row['Hashtags'])):
            if row['Hashtags'][i] in Hillary:
                return 'Hillary'
            elif row['Hashtags'][i] in Trump:
                return 'Trump'
            else:
                continue

    # MENTIONS        
    # Hillary is returned when Hillary is mentioned and something positive 
    # is said or if Trump is mentioned and something negative is said and 
    # Trump the other way around
    if len(row['Mentions']) != 0: 
        if 'HillaryClinton' in row['Mentions'] and 'realDonaldTrump' in row['Mentions']:
                return 'Both'
        for j in range(len(row['Mentions'])):
            if row['Mentions'][j] == "hillaryclinton":
                if row['Positive'] == True:
                    return 'Hillary'
                elif row['Positive'] == False:
                    return 'Trump'
                else:
                    continue
            elif row['Mentions'][j] == "realdonaldtrump":
                if row['Positive'] == True:
                    return 'Trump'
                elif row['Positive'] == False:
                    return 'Hillary'
                else:
                    continue
            else:
                continue
            
    
    # COMBINED
    if len(row['Hashtags']) != 0:
        #print(row)
        for i in range(len(row['Hashtags'])):
            if row['Hashtags'][i] in NeutralHillary:
                if row['Positive'] == True:
                    return 'Hillary'
                elif row['Positive'] == False:
                    return 'Trump'
                else:
                    continue
            if row['Hashtags'][i] in NeutralTrump:
                if row['Positive'] == True:
                    return 'Trump'
                elif row['Positive'] == False:
                    return 'Hillary'
                else:
                    continue
    
    # IF MENTIONS AND HASHTAGS AREN'T CLEAR ENOUGH
    
    if math.isnan(row['quoted_status_id']) == False:
        return 'Quoted'
    elif math.isnan(row['in_reply_to_status_id']) == False:
        return 'Reply'
    else:
        print(row)
        return 'Else'
full['For'] = full.apply(lambda row: trumphillary(row), axis=1)

text                     Hey #ImWithHer🇺🇸👠, and we're making "Herstory" together because we're #StrongerTogether!… https://t.co/HE2lYITEfb                                                                                                                                                                                                              
Positive                 True                                                                                                                                                                                                                                                                                                                           
ID                       296                                                                                                                                                                                                                                                                                          

Name: 1586, dtype: object
text                     #RandPaul #Kentucky=#HillaryClinton-5YrsPrison Lying #Congress Ab HC SALE OF ARMS 2 #Syria!\nhttps://t.co/1cJkkBUagd #LGBtQ #Gays #Hispanics                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                           
Positive                 False                                                                                                                                                                                                                                                               

text                     @shomaristone obvious to everyone w/ open eyes-#DonaldTrump is an ignorant, wildly dishonest, erratic, immature, bullying egomaniac –Krugman                                                                                    
Positive                 False                                                                                                                                                                                                                           
ID                       1994                                                                                                                                                                                                                            
id                       764076921755799554                                                                                                                                                                                                              


text                     BREAKING: Man Who Leaked #HillaryClinton’s Medical Records Found Dead https://t.co/RScxa2CuHi via @peoplesvoice_tv #HillaryClintonBodyCount                                                                                                                                                                                                                                                                                                                                                                         
Positive                 False                                                                                                                                                                                                                                                                                                                                                                                                                                                            

Name: 4928, dtype: object
text                     #MSNBC #TRoberts &amp; guest on how/why #Trump? Face it. All started with hate for #PBO Rs spun lies with msm help. Now you ask why?                                                                                        
Positive                 False                                                                                                                                                                                                                       
ID                       4975                                                                                                                                                                                                                        
id                       764113687036645377                                                                                                                                                                                                 

...

Name: 41955, dtype: object
text                     Tell me the difference between The Clintons in Haiti &amp; D.C. And they looters in Milwaukee?#MAGA #BlackLivesMatter https://t.co/kMDfyq8yU7                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
Positive                 False                                                                                                                                                        

text                     @MADE__USA CloydRivers Hillary that song comes to mind.."🎼 I know you're LYING your lips are moving🎤" WE ARE TIRED ..TURN HER OFF!#MAGA🇺🇸🇺🇸                                                                           
Positive                 True                                                                                                                                                                                                                  
ID                       41841                                                                                                                                                                                                                 
id                       764826284081967104                                                                                                                                                                                                    
CleanText                cloydrivers hil

text                     Deep in ♥of treachery: HC's VP pick TIM KAINE dines w/SOROS' SON as #Trump’s Rise Terrifies World Elite | Breitbart https://t.co/LkqeRpRXVU                                                                                              
Positive                 True                                                                                                                                                                                                                                     
ID                       43983                                                                                                                                                                                                                                    
id                       764850762165972992                                                                                                                                                                                    

Name: 46098, dtype: object
text                     Just saw this in the parking lot at Trader Joe's. Now THAT is how you do sarcasm. 😂 #trump… https://t.co/kdJLi3L8Cr                                                                                                                                                      
Positive                 True                                                                                                                                                                                                                                                                     
ID                       45093                                                                                                                                                                                                                                                                    
id                       764865694680973312                                                         

Name: 47141, dtype: object
text                     Another Dem city lit up... Dumbasses#MAGA                       
Positive                 True                                                            
ID                       46163                                                           
id                       764874636148543488                                              
CleanText                another city dumbassesmaga                                      
entities                 {'hashtags': [], 'urls': [], 'user_mentions': [], 'symbols': []}
in_reply_to_status_id    NaN                                                             
quoted_status_id         NaN                                                             
Mentions                 []                                                              
Hashtags                 []                                                              
Name: 47444, dtype: object


In [34]:
len(full)

48125

In [35]:
full.For.value_counts()

Trump      21492
Hillary    21331
Quoted     4844 
Else       253  
Reply      113  
Both       92   
Name: For, dtype: int64

In [36]:
full.For.value_counts()

Trump      21492
Hillary    21331
Quoted     4844 
Else       253  
Reply      113  
Both       92   
Name: For, dtype: int64

In [37]:
full.in_reply_to_status_id

0       NaN          
1        7.626233e+17
2       NaN          
3       NaN          
4       NaN          
5       NaN          
6        7.640372e+17
7        7.640015e+17
8        7.639497e+17
9       NaN          
10       7.639930e+17
11       7.639060e+17
12      NaN          
13       7.626233e+17
14      NaN          
15       7.639535e+17
16       7.639507e+17
17      NaN          
18       7.637870e+17
19       7.639060e+17
20      NaN          
21      NaN          
22       7.639507e+17
23      NaN          
24       7.639511e+17
25      NaN          
26      NaN          
27      NaN          
28      NaN          
29      NaN          
         ..          
48095    7.648800e+17
48096    7.647999e+17
48097    7.648738e+17
48098    7.648786e+17
48099   NaN          
48100    7.648669e+17
48101    7.648708e+17
48102    7.648669e+17
48103   NaN          
48104    7.648723e+17
48105    7.648329e+17
48106   NaN          
48107   NaN          
48108    7.647999e+17
48109    7

In [38]:
full.For.value_counts()

Trump      21492
Hillary    21331
Quoted     4844 
Else       253  
Reply      113  
Both       92   
Name: For, dtype: int64

In [39]:
for i in range(3):
    print(i)

0
1
2
